In [1]:
import geopandas as gpd
import networkx as nx
from pathlib import Path
import fiona
from libpysal import weights
import momepy
import matplotlib.pyplot as plt
import osmnx as ox
import pandas as pd

roads = r"..\data\UKR_networks_fixed.gpkg"
gdf_roads = gpd.read_file(roads, layer="roads")
print(gdf_roads.crs)
gdf_roads.head(2)

C:\Users\dkerr\Miniconda3\envs\gisrede\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


epsg:6383


,id,length,geometry
0,1,243.672813,"LINESTRING (550265.396 5599118.127, 550269.327..."
1,1,369.206205,"LINESTRING (559190.887 5593222.167, 559196.552..."


In [2]:
G = momepy.gdf_to_nx(gdf_roads, approach="primal")
nodes, edges, sw = momepy.nx_to_gdf(G, points=True, lines=True, spatial_weights=True)

C:\Users\dkerr\Miniconda3\envs\gisrede\lib\site-packages\libpysal\weights\weights.py:224: UserWarning: The weights matrix is not fully connected: 
 There are 218 disconnected components.
  warnings.warn(message)


In [13]:
## FINDING DISCONNECTED COMPONENTS
# Find the disconnected components
components = [x for x in nx.connected_components(G)]
print(len(list(components)))

largest_component = max(components, key=len)
isolated_nodes = [node for component in components if component != largest_component for node in component]
print(f"Nodes in isolated parts of the graph: {len(isolated_nodes)} out of {len(largest_component)}")

218
Nodes in isolated parts of the graph: 797 out of 149856


In [15]:
display(edges.head(3))
nodes.head(3)

,id,length,geometry,mm_len,node_start,node_end
0,1,243.672813,"LINESTRING (550265.396 5599118.127, 550269.327...",243.672813,0,1
1,1,44.894422,"LINESTRING (550233.796 5599149.983, 550250.715...",44.894422,0,536
2,1,21.242812,"LINESTRING (550329.890 5598885.909, 550333.474...",21.242812,1,84404


,nodeID,geometry
0,0,POINT (550265.396 5599118.127)
1,1,POINT (550329.890 5598885.909)
2,2,POINT (559190.887 5593222.167)


In [17]:
edges[edges.duplicated(subset=["node_start", "node_end"])].sort_values(by=["node_start"])

,id,length,geometry,mm_len,node_start,node_end
310,1,92.243678,"LINESTRING (890947.837 5355635.788, 890957.436...",92.243678,141,142
318,1,108.164959,"LINESTRING (890714.533 5357494.565, 890728.214...",108.164959,145,146
501,1,101.371170,"LINESTRING (530625.052 5512159.116, 530613.165...",101.371170,228,229
552,1,108.277055,"LINESTRING (545803.731 5336462.240, 545797.559...",108.277055,252,253
563,1,100.744952,"LINESTRING (545660.301 5330429.530, 545658.484...",100.744952,257,148278
...,...,...,...,...,...,...
175484,1,83.972570,"LINESTRING (330265.968 5608412.710, 330182.167...",83.972570,150438,150439
175493,1,56.671809,"LINESTRING (330167.395 5608392.922, 330224.003...",56.671809,150443,150444
175496,1,94.836135,"LINESTRING (329975.753 5608403.487, 330070.448...",94.836135,150445,150446
175507,1,50.997642,"LINESTRING (330050.664 5608425.150, 330035.557...",50.997642,150454,150455


In [18]:
nodes.to_file("../data/UKR_networks_fixed.gpkg", layer="nodes_ukr_networks", driver="GPKG")

In [19]:
edges.to_file("../data/UKR_networks_fixed.gpkg", layer="edges_ukr_networks", driver="GPKG")

In [25]:
G = nx.MultiGraph()
G.add_nodes_from(nodes.nodeID.unique().tolist())
for index, row in edges.iterrows():
    G.add_edge(row.node_start, row.node_end, weight=row.length)

In [31]:
shortest_path = nx.shortest_path(G, 100681, 63593, weight="weight")
edges_in_path = []
for i in range(len(shortest_path) - 1):
    start_node = shortest_path[i]
    end_node = shortest_path[i + 1]
    edge = edges[
        (edges['node_start'] == start_node) & (edges['node_end'] == end_node) |
        (edges['node_end'] == start_node) & (edges['node_start'] == end_node)
    ]
    edges_in_path.append(edge)
gdf_route = gpd.GeoDataFrame(pd.concat(edges_in_path))
gdf_route.to_file("../data/UKR_networks_fixed.gpkg", layer="route_test", driver="GPKG")

In [32]:
gdf_route.mm_len.sum()

1319610.7170608945

In [ ]:
def calculate_cost(source, target):
    try:
        path = nx.shortest_path(G, source, target, weight='travel_cost')
        cost = nx.shortest_path_length(G, source, target, weight='travel_cost')
        return path, cost
    except nx.NetworkXNoPath:
        return "No path exists", float('inf')
    
source = G.nodes.get((30.50861, 50.50627))
target = G.nodes.get((30.66441, 46.61145))
path, cost = calculate_cost(source, target)
print("Shortest path:", path)
print("Cost:", cost)